In [1]:
import torch
# GPU 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

# CUDA 사용 가능 여부 확인
print(f"CUDA available: {torch.cuda.is_available()}")  # True여야 함

# PyTorch에서 사용하는 CUDA 버전 확인
print(f"CUDA version in PyTorch: {torch.version.cuda}")

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda
NVIDIA GeForce RTX 4050 Laptop GPU
CUDA available: True
CUDA version in PyTorch: 11.8


In [2]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
import matplotlib.pyplot as plt
import torch.nn.functional as F

d:\isy\test\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
import torch
import cv2
import numpy as np
import os
import random
from torchvision import models, transforms

# ResNet 및 Inception 모델 수정
class CustomModel(torch.nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.model = models.efficientnet_b2(weights='IMAGENET1K_V1') 
            self.model.classifier[1] = torch.nn.Linear(self.model.classifier[1].in_features, 1)  # 이진 분류
        
    def forward(self, x):
        return self.model(x)

# efficientnet 모델 로드
efficientnet_model = CustomModel(model_name='efficientnet')
efficientnet_model.eval()  # 평가 모드로 설정

# 전처리 파이프라인 정의
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # OpenCV 이미지를 PIL 이미지로 변환
    transforms.Resize((260, 260)),  # 모델 입력 크기에 맞게 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
])

# 이미지 전처리 함수
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
    image = preprocess(image)  # 전처리 적용
    image = image.unsqueeze(0)  # 배치 차원 추가
    return image

# 예측 함수 정의
def predict_image(model, image_path, device):
    model.to(DEVICE)  # 모델을 디바이스로 이동
    image = preprocess_image(image_path).to(device)  # 전처리 후 디바이스로 이동
    with torch.no_grad():  # 평가 모드에서는 gradient 계산 비활성화
        output = model(image)
        predicted_prob = torch.sigmoid(output).item()  # 확률로 변환

    # 0.5를 기준으로 정상/비정상 예측
    prediction = 'NG (결함 있음)' if predicted_prob > 0.5 else 'OK (정상)'
    return prediction, predicted_prob

# NG 폴더 경로
ng_folder_path = '../PCB_imgs/all/resize/NG/' 
# OK 폴더 경로
ok_folder_path = '../PCB_imgs/all/resize/OK/' 
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 디바이스 설정

# 각 폴더에서 무작위로 10개의 이미지를 가져오기
def load_random_images(folder_path, num_images=10):
    images = os.listdir(folder_path)
    random_images = random.sample(images, min(num_images, len(images)))
    return [os.path.join(folder_path, img) for img in random_images]

# NG 및 OK 이미지 로드
ng_images = load_random_images(ng_folder_path)
ok_images = load_random_images(ok_folder_path)

# NG 이미지 예측 결과와 확률
print(f'NG 예측결과:')
for image_path in ng_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

# OK 이미지 예측 결과와 확률
print(f'OK 예측결과:')
for image_path in ok_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

NG 예측결과:
Efficientnet: OK (정상), 확률: 0.4919
Efficientnet: NG (결함 있음), 확률: 0.5310
Efficientnet: NG (결함 있음), 확률: 0.5072
Efficientnet: OK (정상), 확률: 0.5000
Efficientnet: NG (결함 있음), 확률: 0.5188
Efficientnet: OK (정상), 확률: 0.4762
Efficientnet: NG (결함 있음), 확률: 0.5247
Efficientnet: NG (결함 있음), 확률: 0.5062
Efficientnet: NG (결함 있음), 확률: 0.5311
Efficientnet: OK (정상), 확률: 0.4719
OK 예측결과:
Efficientnet: NG (결함 있음), 확률: 0.6040
Efficientnet: NG (결함 있음), 확률: 0.5677
Efficientnet: NG (결함 있음), 확률: 0.5071
Efficientnet: OK (정상), 확률: 0.4711
Efficientnet: NG (결함 있음), 확률: 0.5034
Efficientnet: OK (정상), 확률: 0.4977
Efficientnet: OK (정상), 확률: 0.4961
Efficientnet: NG (결함 있음), 확률: 0.5232
Efficientnet: NG (결함 있음), 확률: 0.5397
Efficientnet: NG (결함 있음), 확률: 0.5410


In [3]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 260
BATCH_SIZE = 32
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [4]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([32, 3, 260, 260])


In [5]:
# 커스텀 데이터세트 정의
class CustomDataset(Dataset):
    def __init__(self, file_paths, targets, aug=None, preprocess=None):
        self.file_paths = file_paths
        self.targets = targets
        self.aug = aug
        self.preprocess = preprocess

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        target = self.targets[index]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        if self.aug is not None:
            image = self.aug(image=image)['image']

        if self.preprocess is not None:
            image = self.preprocess(image)

        image = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        
        return image, target

In [8]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# 평가 함수 정의
def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            outputs = outputs.view(-1) # torch.Size([batch_size])로 변환
            loss = criterion(outputs, labels)  # 손실 계산
            running_test_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # 0.5 기준으로 이진 분류
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
            
    # 검증 손실과 정확도
    test_loss = running_test_loss / len(test_loader)
    test_accuracy = correct_test / total_test
    return test_loss, test_accuracy

efficientnet_b2 모델 사용

In [19]:
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout1 = nn.Dropout(0.5)  # 첫 번째 드롭아웃
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # 두 번째 드롭아웃
        self.fc2 = nn.Linear(50, 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1408  # EfficientNet B2의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        if isinstance(x, tuple):  
            x = x[0]
        x = x.view(x.size(0), -1)  

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x) 
        x = self.bn1(x) 
        x = self.dropout2(x) 
        x = self.fc2(x)
        
        return x

In [20]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [22]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b2_ft_best_model_01.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.6642, Train Accuracy: 0.5901, Val Loss: 0.4841, Val Accuracy: 0.8108, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.5080, Train Accuracy: 0.7560, Val Loss: 0.4435, Val Accuracy: 0.7988, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.4769, Train Accuracy: 0.7744, Val Loss: 0.3968, Val Accuracy: 0.8406, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.4317, Train Accuracy: 0.8142, Val Loss: 0.3786, Val Accuracy: 0.8466, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.4008, Train Accuracy: 0.8302, Val Loss: 0.3494, Val Accuracy: 0.8805, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.3789, Train Accuracy: 0.8461, Val Loss: 0.3272, Val Accuracy: 0.8865, Learning Rate: 0.000050
Validation loss 

In [23]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b2_ft_best_model_01.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_4440\457214973.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b2_ft_best_mod

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [24]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.1010, Test accuracy: 0.9697


적은 연산량으로 좋은 성능을 낼수있는 모델임.  
feature 8 ~ unfreeze 

In [25]:
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout1 = nn.Dropout(0.5)  # 첫 번째 드롭아웃
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # 두 번째 드롭아웃
        self.fc2 = nn.Linear(50, 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1408  # EfficientNet B2의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        if isinstance(x, tuple):  
            x = x[0]
        x = x.view(x.size(0), -1)  

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x) 
        x = self.bn1(x) 
        x = self.dropout2(x) 
        x = self.fc2(x)
        
        return x

In [26]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [ ]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b2_ft_best_model_02.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.6941, Train Accuracy: 0.5702, Val Loss: 0.5888, Val Accuracy: 0.6793, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.5934, Train Accuracy: 0.6788, Val Loss: 0.5236, Val Accuracy: 0.7709, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.5673, Train Accuracy: 0.6987, Val Loss: 0.4936, Val Accuracy: 0.7809, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.5349, Train Accuracy: 0.7186, Val Loss: 0.4731, Val Accuracy: 0.7948, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.5246, Train Accuracy: 0.7390, Val Loss: 0.4497, Val Accuracy: 0.8127, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.4882, Train Accuracy: 0.7694, Val Loss: 0.4372, Val Accuracy: 0.8207, Learning Rate: 0.000050
Validation loss 

In [ ]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b2_ft_best_model_02.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

model.eval()

In [ ]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

학습률 조정 5e-5 -> 1e-5, features.5~unfreeze

In [ ]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

In [ ]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b2_ft_best_model_03.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

In [ ]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b2_ft_best_model_03.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

model.eval()

In [ ]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')